In [ ]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
from readchar import readchar
import pickle
import cv2
import time

In [ ]:
def main():
	
	env = gym.make('MontezumaRevenge-v0')

	prev_state = env.reset()

	prev_info = {'ale.lives':6}

	env = gym.make('MontezumaRevenge-v0')

	text_model = torch.load('models/sentence/text_model_20')
	image_model = torch.load('models/image/image_model_20')

	with open('dataset/dataset_true.pickle','rb') as f:
		dataset = pickle.load(f)

	with open('first_room.txt','r') as f:
		instructions = f.readlines()

	instructions = [i.strip() for i in instructions]

	print (instructions)

	state = env.reset()

	info = {'ale.lives':6}

	for t in range(100):
		
		action = char_to_action()

		if action == -1:
			env.close()
			break

		state_new, reward, done, info_new = env.step(action)
		
		stack = np.dstack((state,state_new))

		frame_embed = image_model(stack)

		enc_sentence = prepare_sentence(instructions[0], word_to_ix)

		text_embed = text_model(enc_sentence)

		reward = torch.dot(text_embed[0], frame_embed[0]) / (torch.norm(text_embed) * torch.norm(frame_embed))
		
		r = reward.item()

		print (r)

		env.render()

		info = info_new
		state = state_new

	print('Episode %d\t Episode Reward: %f\t'%(episode))

	env.close()

In [ ]:
main()